In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, TimestampType
from datetime import datetime
from pyspark.sql.functions import *
from pyspark.sql.window import Window

# Initialize Spark Session
spark = SparkSession.builder.appName("HospitalTable").getOrCreate()

# -----------------------
# Create Hospital Table
# -----------------------

# Data
hospital_data = [
    (1, 'in', datetime.strptime('2019-12-22 09:00:00', '%Y-%m-%d %H:%M:%S')),
    (1, 'out', datetime.strptime('2019-12-22 09:15:00', '%Y-%m-%d %H:%M:%S')),
    (2, 'in', datetime.strptime('2019-12-22 09:00:00', '%Y-%m-%d %H:%M:%S')),
    (2, 'out', datetime.strptime('2019-12-22 09:15:00', '%Y-%m-%d %H:%M:%S')),
    (2, 'in', datetime.strptime('2019-12-22 09:30:00', '%Y-%m-%d %H:%M:%S')),
    (3, 'out', datetime.strptime('2019-12-22 09:00:00', '%Y-%m-%d %H:%M:%S')),
    (3, 'in', datetime.strptime('2019-12-22 09:15:00', '%Y-%m-%d %H:%M:%S')),
    (3, 'out', datetime.strptime('2019-12-22 09:30:00', '%Y-%m-%d %H:%M:%S')),
    (3, 'in', datetime.strptime('2019-12-22 09:45:00', '%Y-%m-%d %H:%M:%S')),
    (4, 'in', datetime.strptime('2019-12-22 09:45:00', '%Y-%m-%d %H:%M:%S')),
    (5, 'out', datetime.strptime('2019-12-22 09:40:00', '%Y-%m-%d %H:%M:%S'))
]

# Schema
hospital_schema = StructType([
    StructField("emp_id", IntegerType(), True),
    StructField("action", StringType(), True),
    StructField("time", TimestampType(), True)
])

# Create DataFrame
hospital_df = spark.createDataFrame(hospital_data, schema=hospital_schema)

# Create or replace temp view
hospital_df.createOrReplaceTempView("hospital")

# -----------------------
# Check your table!
# -----------------------

spark.sql("SELECT * FROM hospital ORDER BY emp_id, time").show(truncate=False)


+------+------+-------------------+
|emp_id|action|time               |
+------+------+-------------------+
|1     |in    |2019-12-22 09:00:00|
|1     |out   |2019-12-22 09:15:00|
|2     |in    |2019-12-22 09:00:00|
|2     |out   |2019-12-22 09:15:00|
|2     |in    |2019-12-22 09:30:00|
|3     |out   |2019-12-22 09:00:00|
|3     |in    |2019-12-22 09:15:00|
|3     |out   |2019-12-22 09:30:00|
|3     |in    |2019-12-22 09:45:00|
|4     |in    |2019-12-22 09:45:00|
|5     |out   |2019-12-22 09:40:00|
+------+------+-------------------+



In [3]:
#Write a sqk ti fund the total number of perople present in the hospital

In [8]:
spark.sql("""
    select * from (
    select *,
    row_number() over (partition by emp_id order by time desc) as rn
    from hospital)
    where rn = 1 and action = "in"
""").show()

+------+------+-------------------+---+
|emp_id|action|               time| rn|
+------+------+-------------------+---+
|     2|    in|2019-12-22 09:30:00|  1|
|     3|    in|2019-12-22 09:45:00|  1|
|     4|    in|2019-12-22 09:45:00|  1|
+------+------+-------------------+---+



In [16]:
spark.sql("""
    with cte(
        select *,
            case when action = "in" then time end as intime,
            case when action = "out" then time end as outtime
        from hospital)
        
    select emp_id, max(intime), max(outtime)
    from cte
    group by emp_id
    having  max(outtime) < max(intime) or max(outtime) is null
""").show()

+------+-------------------+-------------------+
|emp_id|        max(intime)|       max(outtime)|
+------+-------------------+-------------------+
|     2|2019-12-22 09:30:00|2019-12-22 09:15:00|
|     3|2019-12-22 09:45:00|2019-12-22 09:30:00|
|     4|2019-12-22 09:45:00|               null|
+------+-------------------+-------------------+



In [21]:
int_df = hospital_df. \
    withColumn("intime", when(col("action") == "in", col("time"))). \
    withColumn("outtime", when(col("action") == "out", col("time")))


In [30]:
int_df.groupBy(col("emp_id")).agg(
    max(col("intime")).alias("maxintime"),
    max(col("outtime")).alias("maxouttime"),
).filter((col("maxintime") > col("maxouttime")) | col("maxouttime").isNull()).show()

+------+-------------------+-------------------+
|emp_id|          maxintime|         maxouttime|
+------+-------------------+-------------------+
|     2|2019-12-22 09:30:00|2019-12-22 09:15:00|
|     3|2019-12-22 09:45:00|2019-12-22 09:30:00|
|     4|2019-12-22 09:45:00|               null|
+------+-------------------+-------------------+



In [32]:
(hospital_df.
     groupBy("emp_id").
     pivot("action").
     agg(max("time").alias("time"))).show()
     

+------+-------------------+-------------------+
|emp_id|                 in|                out|
+------+-------------------+-------------------+
|     1|2019-12-22 09:00:00|2019-12-22 09:15:00|
|     2|2019-12-22 09:30:00|2019-12-22 09:15:00|
|     3|2019-12-22 09:45:00|2019-12-22 09:30:00|
|     5|               null|2019-12-22 09:40:00|
|     4|2019-12-22 09:45:00|               null|
+------+-------------------+-------------------+

